# Clustering Identical Neighborhoods as City of Mountain View

### A1.Introduction and Background 
Mountain View is a diverse community nestled between the Santa Cruz Mountains and San Francisco Bay. Which ideally located in the heart of the Silicon Valley (10 miles north of San Jose and 35 miles south of San Francisco). At just over 12 square miles, the City is home to approximately 75,000 residents, many nationally and internationally known corporations, and a thriving small business base. <br>
<br>
Mountain View recognizes the need for a robust local economy and Economic Development continues to be a high priority. The City has been formally recognized as having a solid and diversified tax base; a very strong financial operation; and a low debt burden. <br> <Br>

### A2.Explaination
Here in Mountain View, you get a pretty diverse Asian community, such as Asian linkage restaurants, supermarkets where you can get ethnic Asian food, Asian street food, and beverage shops, etc. Given the fast-paced situation of jobs changing, I might be taking an offer on the other side of the state but which has a similar community nearby. <br><br>
This is how I came up with the idea of clustering other cities in the state which shares identical community and cultures. The information revealed by this report would be useful for new immigrants to California who is looking to the same diversity environments.<br> <br>
Because it helps you to locate the spot where you should be living to gain similar neighborhoods where you have lived.

### B.Data Description
To start with analyzing the data, I would love to scrap resources from below links:
- Scrap html data of Mountain View from wiki page:  https://www.geonames.org/postalcode-search.html?q=Santa+Clara&country=US&adminCode1=CA
- Use foursquare to gain latitude and longitude of neighborhoods of California: https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=CA
- Visialize the neighborhoods of Mountain View by using Folium library
- With foursquare calling, to explore the neighborhoods


### C. The map of Mountain view

In [140]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

done
Sdone

# All requested packages already installed.

Folium installed
Libraries imported.


**C1. Download the data of California from html**

In [141]:
url = 'https://www.geonames.org/postalcode-search.html?q=Santa+Clara&country=US&adminCode1=CA'

res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
tables = soup.select('table')
df_list = []
for table in tables:
    df_list.append(pd.concat(pd.read_html(table.prettify())))

df = df_list[2]
df.head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Santa Clara,95054,United States,California,Santa Clara,NaN
1,NaN,37.392/-121.962,37.392/-121.962,37.392/-121.962,37.392/-121.962,37.392/-121.962,37.392/-121.962
2,2.0,Santa Clara,95050,United States,California,Santa Clara,NaN
3,NaN,37.349/-121.953,37.349/-121.953,37.349/-121.953,37.349/-121.953,37.349/-121.953,37.349/-121.953
4,3.0,Santa Clara,95051,United States,California,Santa Clara,NaN


**C2.Data Preprocessing**

In [142]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Admin3', axis=1, inplace=True)
df.rename(columns={'Admin1':'State','Admin2':'County'}, inplace=True)
df=df[df.index%2==0]

In [143]:
df.reset_index(inplace=True)
df.head()

,index,Place,Code,Country,State,County
0,0,Santa Clara,95054,United States,California,Santa Clara
1,2,Santa Clara,95050,United States,California,Santa Clara
2,4,Santa Clara,95051,United States,California,Santa Clara
3,6,Morgan Hill,95037,United States,California,Santa Clara
4,8,Santa Clara,95052,United States,California,Santa Clara


In [144]:
df.drop('index', axis=1, inplace=True)
df.head()

,Place,Code,Country,State,County
0,Santa Clara,95054,United States,California,Santa Clara
1,Santa Clara,95050,United States,California,Santa Clara
2,Santa Clara,95051,United States,California,Santa Clara
3,Morgan Hill,95037,United States,California,Santa Clara
4,Santa Clara,95052,United States,California,Santa Clara


### D. Importing local csv file conatining the latitudes and longitudes for various neighbourhoods in California given by postal code

**D1. Read data from csv document**

In [145]:
import pandas as pd
lat_lon = pd.read_csv('/Users/alice/Desktop/Coursera/Data Science/2. Python(Pandas)/Applied Data Science Capstone/us-zip-code-latitude-and-longitude.csv')
lat_lon.head()

,Zip,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,92232,33.026203,-115.284581,-8,1,33.026203
1,93227,36.357151,-119.425371,-8,1,36.357151
2,93234,36.209815,-120.084700,-8,1,36.209815
3,93529,37.765218,-119.077690,-8,1,37.765218
4,93761,36.746375,-119.639658,-8,1,36.746375


**D2. Data preprocessing and Cleaning**

In [146]:
lat_lon.drop(['Timezone', 'Daylight savings time flag', 'geopoint'], axis=1, inplace=True)
lat_lon.rename(columns={'Zip':'Code'}, inplace=True)
lat_lon.head()

,Code,Latitude,Longitude
0,92232,33.026203,-115.284581
1,93227,36.357151,-119.425371
2,93234,36.209815,-120.084700
3,93529,37.765218,-119.077690
4,93761,36.746375,-119.639658


In [147]:
#Before merging we need to covert both tables to object
lat_lon = lat_lon.astype(str)
lat_lon.dtypes

Code         object
Latitude     object
Longitude    object
dtype: object

**D3. Merge 2 tables to display zip code, latitue, longitude**

Now that we gained a merged table with information listing countr=y, state, and geographic data, we can start to explore the Neighborhood of Mountainn view!

In [148]:
df2 = pd.merge(df,lat_lon, on="Code")
df2.drop(['Country','State','County'], axis=1,inplace=True)
df2.head()

,Place,Code,Latitude,Longitude
0,Santa Clara,95054,37.393240000000006,-121.96066
1,Santa Clara,95050,37.347791,-121.95131
2,Santa Clara,95051,37.346241,-121.9846
3,Morgan Hill,95037,37.137595000000005,-121.66211000000001
4,Santa Clara,95052,37.189396,-121.705327


### E. Use Foursquare to source neighborhoods

**E1. Import credential of Foursquare**

In [149]:
CLIENT_ID = 'YQCZZIZMAPH51HS0YTNLOZN5GPVJ2EXRP0PDGDOOF1PCU3DP' # your Foursquare ID
CLIENT_SECRET = 'SRXGMUVS2MC4JG44RKONH3VNHKEGXTXCUMK0P031KSCGJXSV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YQCZZIZMAPH51HS0YTNLOZN5GPVJ2EXRP0PDGDOOF1PCU3DP
CLIENT_SECRET:SRXGMUVS2MC4JG44RKONH3VNHKEGXTXCUMK0P031KSCGJXSV


In [150]:
neighborhood_latitude = 37.40679 # neighborhood latitude value
neighborhood_longitude = -122.07461 # neighborhood longitude value
neighborhood_name = 'Mountain View'
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mountain View are 37.40679, -122.07461.


**E2. Use request.get to scrap url information from Foursquare**

In [151]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

import requests # library to handle requests
from pandas.io.json import json_normalize 
results = requests.get(url).json()

In [152]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [153]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,San Veron Park,Park,37.404324,-122.074800
1,"Theatre ""You""",Theater,37.409076,-122.074887
2,Shoreline Blvd & Middlefield Rd,Intersection,37.406097,-122.078100
3,Shell,Gas Station,37.406552,-122.077780
4,Village Lake Apartment Gym,Gym,37.403024,-122.075749


Above we have had the data from Foursquare, was the nearby venues such as park, theater, gas station centroid by Mountain View's location. Let's see how many venues do we have:

In [61]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [154]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
mtv_venues = getNearbyVenues(names=df2['Place'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Santa Clara
Santa Clara
Santa Clara
Morgan Hill
Santa Clara
Santa Clara
Milpitas
Alviso
Sunnyvale
Sunnyvale
Stanford
Campbell
Cupertino
Gilroy
Saratoga
Los Altos
Mountain View
Palo Alto
Palo Alto
Mountain View
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
Santa Clara
Santa Clara
Sunnyvale
Sunnyvale
Los Gatos
Los Gatos
Los Altos
Los Gatos
Mountain View
Palo Alto
Coyote
Redwood Estates
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
Morgan Hill
New Almaden
Mount Hamilton
Campbell
Campbell
Milpitas
San Martin
Cupertino
Gilroy
Los Gatos
Saratoga
Los Altos
Palo Alto
Mountain View
Mountain View
Mountain View
Palo Alto
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jose
San Jo

In [155]:
print('There are {} uniques categories.'.format(len(mtv_venues['Venue Category'].unique())))

There are 211 uniques categories.


We found that there are 211 unique categories of venues, how let's make a metrics of row categories and put 1 if it matchs, put 0 if it does not.<br><br> This Metric will be directly used for clustering.

In [156]:
mtv_onehot = pd.get_dummies(mtv_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mtv_onehot['Neighborhood'] = mtv_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mtv_onehot.columns[-1]] + list(mtv_onehot.columns[:-1])
mtv_onehot = mtv_onehot[fixed_columns]

mtv_onehot.head()

,Neighborhood,ATM,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Santa Clara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Santa Clara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Santa Clara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Santa Clara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Santa Clara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
mtv_grouped = mtv_onehot.groupby('Neighborhood').mean().reset_index()
mtv_grouped

,Neighborhood,ATM,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Alviso,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Campbell,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Coyote,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cupertino,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Gilroy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Holy City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Los Altos,0.083333,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Los Gatos,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.019608,0.019608,0.000000,0.000000
8,Milpitas,0.000000,0.000000,0.013699,0.000000,0.000000,0.000000,0.054795,0.000000,0.000000,...,0.000000,0.013699,0.000000,0.013699,0.041096,0.000000,0.000000,0.000000,0.000000,0.000000
9,Morgan Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000


In [158]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in mtv_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mtv_grouped[mtv_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alviso----
                venue  freq
0  Mexican Restaurant   0.3
1     Harbor / Marina   0.1
2               Trail   0.1
3          Food Truck   0.1
4          Restaurant   0.1


----Campbell----
                venue  freq
0           Gift Shop  0.09
1         Coffee Shop  0.09
2       Grocery Store  0.09
3  Mexican Restaurant  0.04
4          Donut Shop  0.04


----Coyote----
                     venue  freq
0                Gift Shop   1.0
1                      ATM   0.0
2              Pastry Shop   0.0
3               Nail Salon   0.0
4  New American Restaurant   0.0


----Cupertino----
               venue  freq
0    Automotive Shop  0.11
1  Convenience Store  0.11
2        Flea Market  0.11
3               Café  0.11
4        Coffee Shop  0.11


----Gilroy----
                 venue  freq
0       Clothing Store  0.33
1  Sporting Goods Shop  0.17
2                Hotel  0.17
3           Shoe Store  0.08
4        Jewelry Store  0.08


----Holy City----
                     v

**Define a function to sort the venues and convert into dataframe**

In [159]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mtv_grouped['Neighborhood']

for ind in np.arange(mtv_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mtv_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alviso,Mexican Restaurant,Convenience Store,Restaurant,Food Truck,Harbor / Marina,Trail,River,Bar,Flea Market,Falafel Restaurant
1,Campbell,Coffee Shop,Gift Shop,Grocery Store,Discount Store,Gym,Multiplex,Flower Shop,Mexican Restaurant,Breakfast Spot,Park
2,Coyote,Gift Shop,Yoga Studio,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
3,Cupertino,Gym,Café,Convenience Store,Planetarium,Video Store,Gift Shop,Coffee Shop,Flea Market,Automotive Shop,Yoga Studio
4,Gilroy,Clothing Store,Hotel,Sporting Goods Shop,Boutique,Jewelry Store,Shoe Store,Gift Shop,Yoga Studio,Fast Food Restaurant,Food Court


**E3. It's time to cluster Neighborhoods with KMEANS**

In [160]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = mtv_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mtv_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 3], dtype=int32)

In [167]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mtv_merged = df2
#mtv_merged.rename(columns={'Place':'Neighborhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mtv_merged = mtv_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mtv_merged.head() # check the last columns!

,Neighborhood,Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Santa Clara,95054,37.393240000000006,-121.96066,0.0,Gift Shop,Bubble Tea Shop,Liquor Store,Pizza Place,Dive Bar,Shopping Mall,Convenience Store,Motel,Diner,Food Truck
1,Santa Clara,95050,37.347791,-121.95131,0.0,Gift Shop,Bubble Tea Shop,Liquor Store,Pizza Place,Dive Bar,Shopping Mall,Convenience Store,Motel,Diner,Food Truck
2,Santa Clara,95051,37.346241,-121.9846,0.0,Gift Shop,Bubble Tea Shop,Liquor Store,Pizza Place,Dive Bar,Shopping Mall,Convenience Store,Motel,Diner,Food Truck
3,Morgan Hill,95037,37.137595000000005,-121.66211000000001,3.0,Rental Car Location,Warehouse Store,Mexican Restaurant,Yoga Studio,Falafel Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
4,Santa Clara,95052,37.189396,-121.705327,0.0,Gift Shop,Bubble Tea Shop,Liquor Store,Pizza Place,Dive Bar,Shopping Mall,Convenience Store,Motel,Diner,Food Truck


In [170]:
print(mtv_merged[mtv_merged['Cluster Labels'].isnull()])

Neighborhood   Code   Latitude   Longitude  Cluster Labels  \
63  Mount Hamilton  95140  37.388718  -121.63845             NaN   

   1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
63                   NaN                   NaN                   NaN   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
63                   NaN                   NaN                   NaN   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
63                   NaN                   NaN                   NaN   

   10th Most Common Venue  
63                    NaN  


In [172]:
pd.to_numeric(mtv_merged['Cluster Labels'], errors='coerce')
mtv_merged = mtv_merged.dropna(subset=['Cluster Labels'])
mtv_merged['Cluster Labels']=mtv_merged['Cluster Labels'].astype(int)

**E4. The final map of clustering cities in California based on Mountain View**

In [173]:
map_clusters = folium.Map(location=[37.40679,-122.07461],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(mtv_merged['Latitude'], mtv_merged['Longitude'], mtv_merged['Neighborhood'], mtv_merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Conclusion and Discussion
In this report I have used KMeans to cluster adjacent cities in California, no matter which city you are relocating to, you can find the optimun spot where you get adequately affinity community. <br> <br>
For instance, if you are moving to San Jose, you may enjoy living in Pedro Street.